# Database Setup - Initialize All Databases

## Purpose

This notebook creates and populates all SQLite databases from source Excel files:

1. **content_repo.db** - Content repository (from content_repo.csv.xlsx)
2. **faq_question.db** - FAQ questions (from faq_question.csv.xlsx)
3. **faq_answer.db** - FAQ answers (from faq_answer.csv.xlsx)
4. **version_tracking.db** - Version history tracking (schema only)

## When to Run

- **First time setup**: Initial database creation
- **Data refresh**: When Excel files are updated
- **Reset databases**: Clean slate for testing

## Directory Structure

```
FAQ/
├── databases/              ← All SQLite databases here
│   ├── content_repo.db
│   ├── faq_question.db
│   ├── faq_answer.db
│   └── version_tracking.db
├── Docs/                   ← Source Excel files
│   ├── content_repo.csv.xlsx
│   ├── faq_question.csv.xlsx
│   └── faq_answer.csv.xlsx
└── notebooks/
    └── setup_databases.ipynb  ← This notebook
```

## Step 1: Setup and Configuration

In [ ]:
import pandas as pd
import sqlite3
from pathlib import Path
import sys

# ============================================================================
# DATA SOURCE CONFIGURATION - CHANGE THIS TO SWITCH SOURCES
# ============================================================================

DATA_SOURCE = 'excel'  # Options: 'excel' or 'test'

# ============================================================================
# PATH CONFIGURATION
# ============================================================================

DOCS_PATH = Path('../sql')
DATABASES_DIR = Path('../databases')

# Source Excel files (used when DATA_SOURCE='excel')
CONTENT_REPO_XLSX = DOCS_PATH / 'content_repo.xlsx'
FAQ_QUESTION_XLSX = DOCS_PATH / 'faq_question.xlsx'
FAQ_ANSWER_XLSX = DOCS_PATH / 'faq_answer.xlsx'

# Database paths
CONTENT_REPO_DB = DATABASES_DIR / 'content_repo.db'
FAQ_QUESTION_DB = DATABASES_DIR / 'faq_question.db'
FAQ_ANSWER_DB = DATABASES_DIR / 'faq_answer.db'
VERSION_TRACKING_DB = DATABASES_DIR / 'version_tracking.db'

# Schema file
SCHEMA_SQL = DOCS_PATH / 'create_schema.sql'

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Create databases directory
DATABASES_DIR.mkdir(parents=True, exist_ok=True)

print("=" * 80)
print(f"📊 DATABASE SETUP - Data Source: {DATA_SOURCE.upper()}")
print("=" * 80)
print(f"\n📁 Paths:")
print(f"  Source: {DOCS_PATH}")
print(f"  Databases: {DATABASES_DIR}")
print(f"\n💡 To switch data source, change DATA_SOURCE variable above")
print(f"   - 'excel': Load from Excel files (production)")
print(f"   - 'test': Generate test data (examples)")
print("=" * 80)

## Step 2: Data Loading Functions

In [ ]:
import hashlib
from datetime import datetime, timedelta

# ============================================================================
# EXCEL DATA LOADERS
# ============================================================================

def load_content_repo_excel():
    """Load content_repo from Excel file"""
    print(f"📥 Loading {CONTENT_REPO_XLSX.name}...")
    df = pd.read_excel(CONTENT_REPO_XLSX)
    print(f"  ✅ Loaded {len(df)} records")
    return df

def load_faq_question_excel():
    """Load faq_question from Excel file"""
    print(f"📥 Loading {FAQ_QUESTION_XLSX.name}...")
    df = pd.read_excel(FAQ_QUESTION_XLSX)
    print(f"  ✅ Loaded {len(df)} records")
    return df

def load_faq_answer_excel():
    """Load faq_answer from Excel file"""
    print(f"📥 Loading {FAQ_ANSWER_XLSX.name}...")
    df = pd.read_excel(FAQ_ANSWER_XLSX)
    print(f"  ✅ Loaded {len(df)} records")
    return df

# ============================================================================
# TEST DATA GENERATORS
# ============================================================================

def generate_test_content_repo():
    """Generate test data for content_repo (3 pages - simple initial state)"""
    print("🔧 Generating test data for content_repo...")
    
    # Test data directory
    test_data_dir = Path('../test_data_version_tracking')
    markdown_dir = test_data_dir / 'markdown_files'
    markdown_dir.mkdir(parents=True, exist_ok=True)
    
    # Policy content
    policy_content = {
        'page1_v1': """# Employee Leave Policy

## 1. Purpose
This policy establishes guidelines for employee leave requests and approvals.

## 2. Scope
This policy applies to all full-time employees.

## 3. Annual Leave
- Employees accrue 15 days of annual leave per year
- Leave must be requested at least 2 weeks in advance
- Maximum carryover: 5 days to next year
""",
        'page2_v1': """## 4. Sick Leave
Employees are entitled to sick leave as follows:
- 10 days of paid sick leave per year
- Medical certificate required for absences over 3 consecutive days
- Unused sick leave does not carry over

## 5. Parental Leave
- 12 weeks of paid parental leave for primary caregiver
- 4 weeks of paid parental leave for secondary caregiver
""",
        'page3_v1': """## 6. Bereavement Leave
- 5 days for immediate family member
- 2 days for extended family member
- No documentation required

## 7. Leave Request Process
1. Submit request via HR portal
2. Manager approval required
3. Confirmation sent within 48 hours
"""
    }
    
    def calculate_checksum(content):
        return hashlib.sha256(content.encode('utf-8')).hexdigest()
    
    def create_markdown_file(file_path, content):
        file_path.parent.mkdir(parents=True, exist_ok=True)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        return calculate_checksum(content)
    
    # Timeline
    base_time = datetime(2025, 1, 1, 10, 0, 0)
    def get_timestamp(days_offset, hours_offset=0):
        dt = base_time + timedelta(days=days_offset, hours=hours_offset)
        return dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    
    # Create records
    records = []
    
    # Page 1
    md_path_p1 = markdown_dir / 'leave_policy' / 'page1_v1.md'
    checksum_p1 = create_markdown_file(md_path_p1, policy_content['page1_v1'])
    
    records.append({
        'ud_source_file_id': 1001,
        'domain': 'HR',
        'service': 'Policy Management',
        'orgoid': 'ORG001',
        'associateoid': 'ASSOC001',
        'raw_file_nme': 'Employee_Leave_Policy.pdf',
        'raw_file_type': '.pdf',
        'raw_file_version_nbr': 1,
        'source_url_txt': None,
        'parent_location_txt': None,
        'raw_file_path': '/test/Employee_Leave_Policy.pdf',
        'extracted_markdown_file_path': str(md_path_p1.absolute()),
        'extracted_layout_file_path': 'layout_p1.txt',
        'raw_file_page_nbr': 1,
        'title_nme': 'Employee Leave Policy',
        'breadcrumb_txt': None,
        'content_tags_txt': None,
        'version_nbr': 1,
        'content_checksum': checksum_p1,
        'file_status': 'Active',
        'created_dt': get_timestamp(0),
        'last_modified_dt': get_timestamp(0)
    })
    
    # Page 2
    md_path_p2 = markdown_dir / 'leave_policy' / 'page2_v1.md'
    checksum_p2 = create_markdown_file(md_path_p2, policy_content['page2_v1'])
    
    records.append({
        'ud_source_file_id': 1002,
        'domain': 'HR',
        'service': 'Policy Management',
        'orgoid': 'ORG001',
        'associateoid': 'ASSOC001',
        'raw_file_nme': 'Employee_Leave_Policy.pdf',
        'raw_file_type': '.pdf',
        'raw_file_version_nbr': 1,
        'source_url_txt': None,
        'parent_location_txt': None,
        'raw_file_path': '/test/Employee_Leave_Policy.pdf',
        'extracted_markdown_file_path': str(md_path_p2.absolute()),
        'extracted_layout_file_path': 'layout_p2.txt',
        'raw_file_page_nbr': 2,
        'title_nme': 'Employee Leave Policy',
        'breadcrumb_txt': None,
        'content_tags_txt': None,
        'version_nbr': 1,
        'content_checksum': checksum_p2,
        'file_status': 'Active',
        'created_dt': get_timestamp(0),
        'last_modified_dt': get_timestamp(0)
    })
    
    # Page 3
    md_path_p3 = markdown_dir / 'leave_policy' / 'page3_v1.md'
    checksum_p3 = create_markdown_file(md_path_p3, policy_content['page3_v1'])
    
    records.append({
        'ud_source_file_id': 1003,
        'domain': 'HR',
        'service': 'Policy Management',
        'orgoid': 'ORG001',
        'associateoid': 'ASSOC001',
        'raw_file_nme': 'Employee_Leave_Policy.pdf',
        'raw_file_type': '.pdf',
        'raw_file_version_nbr': 1,
        'source_url_txt': None,
        'parent_location_txt': None,
        'raw_file_path': '/test/Employee_Leave_Policy.pdf',
        'extracted_markdown_file_path': str(md_path_p3.absolute()),
        'extracted_layout_file_path': 'layout_p3.txt',
        'raw_file_page_nbr': 3,
        'title_nme': 'Employee Leave Policy',
        'breadcrumb_txt': None,
        'content_tags_txt': None,
        'version_nbr': 1,
        'content_checksum': checksum_p3,
        'file_status': 'Active',
        'created_dt': get_timestamp(0),
        'last_modified_dt': get_timestamp(0)
    })
    
    df = pd.DataFrame(records)
    print(f"  ✅ Generated {len(df)} content records (3 pages)")
    return df

def generate_test_faq_question():
    """Generate comprehensive test data for faq_question (covering all pages)"""
    print("🔧 Generating test data for faq_question...")
    
    now = datetime.now().isoformat()
    
    records = [
        # Page 1 questions (Annual Leave)
        {
            'question_id': 'Q1001',
            'prev_recommended_question_txt': None,
            'prev_question_txt': None,
            'recommeded_question_txt': 'What is the annual leave policy?',
            'question_txt': 'What is the annual leave policy?',
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1001',
            'src_page_number': 1,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'question_id': 'Q1002',
            'prev_recommended_question_txt': None,
            'prev_question_txt': None,
            'recommeded_question_txt': 'How many annual leave days do I get per year?',
            'question_txt': 'How many annual leave days do I get per year?',
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1001',
            'src_page_number': 1,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'question_id': 'Q1003',
            'prev_recommended_question_txt': None,
            'prev_question_txt': None,
            'recommeded_question_txt': 'Can I carry over unused annual leave to next year?',
            'question_txt': 'Can I carry over unused annual leave to next year?',
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1001',
            'src_page_number': 1,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        
        # Page 2 questions (Sick Leave & Parental Leave)
        {
            'question_id': 'Q1004',
            'prev_recommended_question_txt': None,
            'prev_question_txt': None,
            'recommeded_question_txt': 'How many sick leave days do employees get?',
            'question_txt': 'How many sick leave days do employees get?',
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1002',
            'src_page_number': 2,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'question_id': 'Q1005',
            'prev_recommended_question_txt': None,
            'prev_question_txt': None,
            'recommeded_question_txt': 'When do I need a medical certificate for sick leave?',
            'question_txt': 'When do I need a medical certificate for sick leave?',
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1002',
            'src_page_number': 2,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'question_id': 'Q1006',
            'prev_recommended_question_txt': None,
            'prev_question_txt': None,
            'recommeded_question_txt': 'What is the parental leave policy?',
            'question_txt': 'What is the parental leave policy?',
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1002',
            'src_page_number': 2,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'question_id': 'Q1007',
            'prev_recommended_question_txt': None,
            'prev_question_txt': None,
            'recommeded_question_txt': 'How much parental leave do primary caregivers get?',
            'question_txt': 'How much parental leave do primary caregivers get?',
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1002',
            'src_page_number': 2,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        
        # Page 3 questions (Bereavement & Leave Process)
        {
            'question_id': 'Q1008',
            'prev_recommended_question_txt': None,
            'prev_question_txt': None,
            'recommeded_question_txt': 'What is the bereavement leave policy?',
            'question_txt': 'What is the bereavement leave policy?',
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1003',
            'src_page_number': 3,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'question_id': 'Q1009',
            'prev_recommended_question_txt': None,
            'prev_question_txt': None,
            'recommeded_question_txt': 'How do I request leave?',
            'question_txt': 'How do I request leave?',
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1003',
            'src_page_number': 3,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'question_id': 'Q1010',
            'prev_recommended_question_txt': None,
            'prev_question_txt': None,
            'recommeded_question_txt': 'How long does it take to get leave approval?',
            'question_txt': 'How long does it take to get leave approval?',
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1003',
            'src_page_number': 3,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        }
    ]
    
    df = pd.DataFrame(records)
    print(f"  ✅ Generated {len(df)} FAQ questions (covering all 3 pages)")
    return df

def generate_test_faq_answer():
    """Generate comprehensive test data for faq_answer (matching all questions)"""
    print("🔧 Generating test data for faq_answer...")
    
    now = datetime.now().isoformat()
    
    records = [
        # Answers for Page 1 questions
        {
            'answer_id': 'A1001',
            'question_id': 'Q1001',
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Employees accrue 15 days of annual leave per year.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Employees accrue 15 days of annual leave per year. Leave must be requested at least 2 weeks in advance, and a maximum of 5 days can be carried over to the next year.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1001',
            'src_page_number': 1,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 'A1002',
            'question_id': 'Q1002',
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Employees accrue 15 days annually.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'All full-time employees accrue 15 days of annual leave per year.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1001',
            'src_page_number': 1,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 'A1003',
            'question_id': 'Q1003',
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Yes, up to 5 days can be carried over.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Yes, you can carry over unused annual leave to the next year, with a maximum carryover of 5 days.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1001',
            'src_page_number': 1,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        
        # Answers for Page 2 questions
        {
            'answer_id': 'A1004',
            'question_id': 'Q1004',
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Employees receive 10 days of paid sick leave per year.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Employees are entitled to 10 days of paid sick leave per year. A medical certificate is required for absences over 3 consecutive days. Unused sick leave does not carry over.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1002',
            'src_page_number': 2,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 'A1005',
            'question_id': 'Q1005',
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Medical certificate required for absences over 3 consecutive days.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'A medical certificate is required for sick leave absences that exceed 3 consecutive days.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1002',
            'src_page_number': 2,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 'A1006',
            'question_id': 'Q1006',
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': '12 weeks paid for primary caregiver, 4 weeks for secondary.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Primary caregivers receive 12 weeks of paid parental leave, while secondary caregivers receive 4 weeks of paid parental leave.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1002',
            'src_page_number': 2,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 'A1007',
            'question_id': 'Q1007',
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': '12 weeks of paid parental leave.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Primary caregivers are entitled to 12 weeks of paid parental leave.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1002',
            'src_page_number': 2,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        
        # Answers for Page 3 questions
        {
            'answer_id': 'A1008',
            'question_id': 'Q1008',
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': '5 days for immediate family, 2 days for extended family.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Bereavement leave provides 5 days for immediate family members and 2 days for extended family members. No documentation is required.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1003',
            'src_page_number': 3,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 'A1009',
            'question_id': 'Q1009',
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Submit via HR portal and get manager approval.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'To request leave: 1) Submit request via HR portal, 2) Obtain manager approval, 3) Receive confirmation within 48 hours.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1003',
            'src_page_number': 3,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 'A1010',
            'question_id': 'Q1010',
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Confirmation is sent within 48 hours.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Leave approval confirmation is sent within 48 hours of submission, after manager approval.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1003',
            'src_page_number': 3,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        }
    ]
    
    df = pd.DataFrame(records)
    print(f"  ✅ Generated {len(df)} FAQ answers (matching all questions)")
    return df

# ============================================================================
# UNIFIED DATA LOADERS
# ============================================================================

def load_data(data_type):
    """Load data based on configured source"""
    if DATA_SOURCE == 'excel':
        if data_type == 'content_repo':
            return load_content_repo_excel()
        elif data_type == 'faq_question':
            return load_faq_question_excel()
        elif data_type == 'faq_answer':
            return load_faq_answer_excel()
    elif DATA_SOURCE == 'test':
        if data_type == 'content_repo':
            return generate_test_content_repo()
        elif data_type == 'faq_question':
            return generate_test_faq_question()
        elif data_type == 'faq_answer':
            return generate_test_faq_answer()
    else:
        raise ValueError(f"Invalid DATA_SOURCE: {DATA_SOURCE}. Must be 'excel' or 'test'")

print("✅ Data loading functions defined")

## Step 3: Load All Data

In [ ]:
print(f"\n📥 Loading data from {DATA_SOURCE.upper()} source...\n")

# Load all datasets
content_repo = load_data('content_repo')
faq_question = load_data('faq_question')
faq_answer = load_data('faq_answer')

print(f"\n✅ All data loaded successfully")
print(f"\n📊 Summary:")
print(f"  - content_repo: {len(content_repo)} records")
print(f"  - faq_question: {len(faq_question)} records")
print(f"  - faq_answer: {len(faq_answer)} records")

## Step 4: Create Databases

In [ ]:
def create_database(db_path, table_name, data, display_cols=None):
    """Unified database creation function"""
    print(f"\n📊 Creating {db_path.name}...")
    
    # Remove existing database
    if db_path.exists():
        try:
            db_path.unlink()
            print(f"  🗑️ Removed existing database")
        except PermissionError:
            print(f"  ⚠️ Cannot remove existing database (file in use)")
            print(f"  💡 Close any connections to {db_path.name} and try again")
            return None
    
    # Create database
    conn = sqlite3.connect(db_path)
    data.to_sql(table_name, conn, if_exists='replace', index=False)
    
    # Verify
    count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table_name}", conn).iloc[0]['count']
    print(f"  ✅ Created {table_name} table with {count} records")
    
    # Show sample
    sample = pd.read_sql(f"SELECT * FROM {table_name} LIMIT 3", conn)
    if display_cols:
        print(f"\n📄 Sample records:")
        print(sample[display_cols].to_string(index=False))
    
    conn.close()
    print(f"✅ {db_path.name} created successfully")
    
    return True

# Create all databases
print("=" * 80)
print("CREATING DATABASES")
print("=" * 80)

# content_repo.db
create_database(
    CONTENT_REPO_DB, 
    'content_repo', 
    content_repo,
    display_cols=['ud_source_file_id', 'raw_file_nme', 'raw_file_page_nbr', 'file_status']
)

# faq_question.db
result = create_database(
    FAQ_QUESTION_DB, 
    'faq_questions', 
    faq_question,
    display_cols=['question_id', 'question_txt', 'domain']
)

# faq_answer.db
create_database(
    FAQ_ANSWER_DB, 
    'faq_answers', 
    faq_answer,
    display_cols=['answer_id', 'question_id', 'domain']
)

## Step 5: Create version_tracking.db (Schema Only)

In [ ]:
print(f"\n📊 Creating {VERSION_TRACKING_DB.name}...")

# Remove existing database
if VERSION_TRACKING_DB.exists():
    try:
        VERSION_TRACKING_DB.unlink()
        print("  🗑️ Removed existing database")
    except PermissionError:
        print("  ⚠️ Cannot remove existing database (file in use)")
        print("  💡 Close any connections to version_tracking.db and try again")

# Create database if removed successfully or doesn't exist
if not VERSION_TRACKING_DB.exists():
    conn_version = sqlite3.connect(VERSION_TRACKING_DB)
    
    # Read and execute schema
    if SCHEMA_SQL.exists():
        print(f"  📋 Loading schema from {SCHEMA_SQL.name}...")
        with open(SCHEMA_SQL, 'r', encoding='utf-8') as f:
            schema_sql = f.read()
        
        try:
            conn_version.executescript(schema_sql)
            conn_version.commit()
            print("  ✅ Schema executed successfully")
            
            # Verify tables
            tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn_version)
            print(f"  ✅ Created {len(tables)} tables:")
            for table in tables['name']:
                count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", conn_version).iloc[0]['count']
                print(f"    - {table}: {count} records (empty - will be populated by pipeline)")
        except sqlite3.OperationalError as e:
            print(f"  ⚠️ Error executing schema: {e}")
    else:
        print(f"  ⚠️ Schema file not found: {SCHEMA_SQL}")
    
    conn_version.close()
    print(f"✅ {VERSION_TRACKING_DB.name} created successfully")

## Step 6: Summary and Verification

In [ ]:
print("\n" + "="*80)
print(f"📊 DATABASE SETUP SUMMARY - Source: {DATA_SOURCE.upper()}")
print("="*80)

databases = [
    (CONTENT_REPO_DB, 'content_repo', 'Content repository'),
    (FAQ_QUESTION_DB, 'faq_questions', 'FAQ questions'),
    (FAQ_ANSWER_DB, 'faq_answers', 'FAQ answers'),
    (VERSION_TRACKING_DB, None, 'Version tracking')
]

print(f"\n📁 Database Directory: {DATABASES_DIR}\n")

total_size = 0
summary_data = []

for db_path, table_name, description in databases:
    if db_path.exists():
        size_kb = db_path.stat().st_size / 1024
        total_size += size_kb
        
        # Get record count
        if table_name:
            conn = sqlite3.connect(db_path)
            count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table_name}", conn).iloc[0]['count']
            conn.close()
            status = f"{count:,} records"
        else:
            conn = sqlite3.connect(db_path)
            tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
            conn.close()
            status = f"{len(tables)} tables created"
        
        summary_data.append({
            'Database': db_path.name,
            'Description': description,
            'Size (KB)': f"{size_kb:.1f}",
            'Status': status,
            'Source': DATA_SOURCE
        })
        
        print(f"✅ {db_path.name}")
        print(f"   {description}")
        print(f"   Size: {size_kb:.1f} KB | Status: {status}")
        print()
    else:
        summary_data.append({
            'Database': db_path.name,
            'Description': description,
            'Size (KB)': 'N/A',
            'Status': 'NOT FOUND',
            'Source': DATA_SOURCE
        })
        print(f"❌ {db_path.name} - NOT FOUND")
        print()

# Display summary table
summary_df = pd.DataFrame(summary_data)

print(f"\n📊 Total database size: {total_size:.1f} KB ({total_size/1024:.2f} MB)")
print("\n" + "="*80)
print("✅ DATABASE SETUP COMPLETED SUCCESSFULLY!")
print("="*80)

print(f"\n📋 Summary Table:")
print(summary_df.to_string(index=False))

print(f"\n💡 Next Steps:")
print("  1. Run daily_version_tracking_pipeline.ipynb to populate version history")
print("  2. Use version_tracking_implementation.ipynb to explore version chains")
print("  3. All pipelines will automatically use databases from 'databases/' folder")

print(f"\n🔄 To Switch Data Source:")
print(f"  - Change DATA_SOURCE variable in Step 1")
print(f"  - Run all cells again")
print(f"  - Current source: {DATA_SOURCE.upper()}")

## Database Connection Information

### For Other Notebooks

Use these connection patterns in other notebooks:

```python
import sqlite3
import pandas as pd
from pathlib import Path

# Database paths
DATABASES_DIR = Path('../databases')
CONTENT_REPO_DB = DATABASES_DIR / 'content_repo.db'
FAQ_QUESTION_DB = DATABASES_DIR / 'faq_question.db'
FAQ_ANSWER_DB = DATABASES_DIR / 'faq_answer.db'
VERSION_TRACKING_DB = DATABASES_DIR / 'version_tracking.db'

# Connect to databases
conn_content = sqlite3.connect(CONTENT_REPO_DB)
conn_question = sqlite3.connect(FAQ_QUESTION_DB)
conn_answer = sqlite3.connect(FAQ_ANSWER_DB)
conn_version = sqlite3.connect(VERSION_TRACKING_DB)

# Load data
content_repo = pd.read_sql("SELECT * FROM content_repo", conn_content)
faq_questions = pd.read_sql("SELECT * FROM faq_questions", conn_question)
faq_answers = pd.read_sql("SELECT * FROM faq_answers", conn_answer)

# Close connections when done
conn_content.close()
conn_question.close()
conn_answer.close()
conn_version.close()
```

### Data Source Information

**Current Data Source:** The databases were created from the `{DATA_SOURCE}` source.

**To verify source:**
- Check the 'source' column in faq_question or faq_answer tables
- Test data will have source='test_data'
- Excel data will have source from original spreadsheets

## Step 8: Summary and Verification

## Database Connection Information

### For Other Notebooks

Use these connection patterns in other notebooks:

```python
import sqlite3
from pathlib import Path

# Database paths
DATABASES_DIR = Path('../databases')
CONTENT_REPO_DB = DATABASES_DIR / 'content_repo.db'
FAQ_QUESTION_DB = DATABASES_DIR / 'faq_question.db'
FAQ_ANSWER_DB = DATABASES_DIR / 'faq_answer.db'
VERSION_TRACKING_DB = DATABASES_DIR / 'version_tracking.db'

# Connect to databases
conn_content = sqlite3.connect(CONTENT_REPO_DB)
conn_question = sqlite3.connect(FAQ_QUESTION_DB)
conn_answer = sqlite3.connect(FAQ_ANSWER_DB)
conn_version = sqlite3.connect(VERSION_TRACKING_DB)

# Load data
content_repo = pd.read_sql("SELECT * FROM content_repo", conn_content)
faq_questions = pd.read_sql("SELECT * FROM faq_questions", conn_question)
faq_answers = pd.read_sql("SELECT * FROM faq_answers", conn_answer)

# Close connections when done
conn_content.close()
conn_question.close()
conn_answer.close()
conn_version.close()
```